## PDF De-identification Pipeline
This Pretrained Pipeline will de-identify PDFs documents, and write results back to PDF.

In [2]:
import json
import os
import shutil
import sparkocr

from pyspark.sql import SparkSession
from pyspark import SparkConf


params = {"spark.driver.memory":"20G",
          "spark.executor.memory":"20G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M",
          "spark.sql.legacy.allowUntypedScalaUDF":True}


spark = sparkocr.start(jar_path="../../target/scala-2.12")
spark

Spark version: 3.4.0
Spark NLP version: 5.4.0
Spark NLP for Healthcare version: 5.3.2
Spark OCR version: 5.4.0




## Load Sample Data

In [3]:
file_path='./mr_simpson.pdf'
pdfs = spark.read.format("binaryFile").load(file_path)

## Run from PretrainedPipeline

In [4]:
from sparknlp.pretrained import *
pipeline = PretrainedPipeline("pdf_deid_subentity_context_augmented_pipeline", "en", "clinical/models")

pdf_deid_subentity_context_augmented_pipeline download started this may take some time.
Approx size to download 1.6 GB
[OK!]


## Write PDF to local storage

In [8]:
%%time
result = pipeline.transform(pdfs)
outputPath = "./"
for row in result.select("pdf", "path").toLocalIterator():
    inputFile = row.path
    fileName = inputFile.split("/")[-1].split(".")[0] + "_deIdentified.pdf"
    outputFilePath = outputPath + fileName

    print(f"Saving file to {outputFilePath}")
    with open(outputFilePath, "wb") as f:
        f.write(row.pdf)
    

Saving file to ./mr_simpson_deIdentified.pdf
CPU times: user 88 ms, sys: 10.7 ms, total: 98.8 ms
Wall time: 3.39 s


In [10]:
from sparkocr.utils import display_pdf_file
display_pdf_file(fileName)